In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re

1) Log file is loaded

In [2]:
file = open("log.txt", "r") 
lines = file.readlines()
for l in range(3): print(lines[l][:-1])

156.15.13.173 [01/Jan/2018:11:13:49 +0100] "GET AddProduct.html HTTP/1.0" 200 2356
176.247.135.49 [01/Jan/2018:11:18:07 +0100] "GET Shop.html HTTP/1.0" 200 7466
176.247.135.49 [01/Jan/2018:11:19:05 +0100] "GET AddProduct.html HTTP/1.0" 200 2356


2) Class "Request" parses and keeps data of a single requests. Complete the TODO, where a given "line" (string) should be parsed in order to dervie the ip of the host making the request, time of the request (in seconds), and target page (name).

In [97]:
date_regex_obj = re.compile(r'(?P<day>[0-9]{2})/(?P<month>\w+)/(?P<year>[0-9]{4}):(?P<hour>[0-9]{2}):(?P<minute>[0-9]{2}):(?P<second>[0-9]{2})\s(?P<utc_offset>[-+0-9]+)')

def fix_date(string):
    result = date_regex_obj.match(string)
    second = int(result.group('second'))
    minute = int(result.group('minute'))
    hour = int(result.group('hour'))
    day = int(result.group('day'))
    if second >= 60:
        minute += 1
        second -= 60
    if minute >= 60:
        hour += 1
        minute -= 60
    if hour >= 24:
        day += 1
    date = datetime.datetime.strptime('{:02d}/{}/{}:{:02d}:{:02d}:{:02d} {}'.format(day, result.group('month'), result.group('year'), hour, minute, second, result.group('utc_offset')), '%d/%b/%Y:%H:%M:%S %z')
    return date


In [104]:
class Request:
    def __init__(self, line):
        # TODO DO PARSING HERE
        ip, tmp = line.split(' ', 1)
        # Because time strptime doesn't always work (when minutes or seconds = 60)
        pattern = r'(?P<ip>[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\s\[(?P<date_time>.+)\]\s\"\w+\s(?P<site>[a-zA-Z\.]+)'
        match_res = re.match(pattern, line)
        if match_res:
            self.ip = match_res.group('ip')
            # convert hours + minutes + seconds -> total number of seconds
            try:
                time_tmp = datetime.datetime.strptime(match_res.group('date_time'), '%d/%b/%Y:%H:%M:%S %z')
            except ValueError:  # Sometimes seconds or minutes are equal to 60 in the log file
                time_tmp = fix_date(match_res.group('date_time'))
            hour = time_tmp.hour
            minute = time_tmp.minute
            second = time_tmp.second
            self.time = second + 60*minute + 3600*hour
            self.page = match_res.group('site')
        else:
            print('Error while parsing line. Match not found.')
        
    def __repr__(self):
        return self.ip + "     " +  str(datetime.timedelta(seconds=self.time)) + " -> " + self.page

3) Each line of log.txt is parsed and the Request objects are added to the list

In [105]:
reqs = [Request(line) for line in lines]
for l in range(10): print(reqs[l) 

156.15.13.173     11:13:49 -> AddProduct.html 156.15.13.173 [01/Jan/2018:11:13:49 +0100] "GET AddProduct.html HTTP/1.0" 200 2356

176.247.135.49     11:18:07 -> Shop.html 176.247.135.49 [01/Jan/2018:11:18:07 +0100] "GET Shop.html HTTP/1.0" 200 7466

176.247.135.49     11:19:05 -> AddProduct.html 176.247.135.49 [01/Jan/2018:11:19:05 +0100] "GET AddProduct.html HTTP/1.0" 200 2356

176.247.135.49     11:29:05 -> Summary.html 176.247.135.49 [01/Jan/2018:11:29:05 +0100] "GET Summary.html HTTP/1.0" 200 2693

69.84.30.199     11:43:59 -> Shop.html 69.84.30.199 [01/Jan/2018:11:43:59 +0100] "GET Shop.html HTTP/1.0" 200 7466

69.84.30.199     11:45:16 -> AddProduct.html 69.84.30.199 [01/Jan/2018:11:45:16 +0100] "GET AddProduct.html HTTP/1.0" 200 2356

57.116.45.120     11:46:45 -> AddProduct.html 57.116.45.120 [01/Jan/2018:11:46:45 +0100] "GET AddProduct.html HTTP/1.0" 200 2356

69.84.30.199     11:50:36 -> Summary.html 69.84.30.199 [01/Jan/2018:11:50:36 +0100] "GET Summary.html HTTP/1.0" 200 26

# Exercise

Your task is to analyze the log file. You should indentify the users and sessions. Then, compute the following: <br>
3.1) the number of sessions (should be 1000 :) ), <br>
3.2) the number of users, <br>
3.3) the average number of sessions per one user <br>
3.4) use the distribution plot to visualize the sessions' lengths (minutes);  <br>
3.5) use the distribution plot to visualize the sessions' lengths (the number of requests) <br>
3.6) use the distribution plot to visualize sessions' starting times (hours). <br>
3.7) which web page is the most common entry point?; for each page plot the number of times it has been an entry page,  <br>
3.8) which web page is the most common departure point?; for each page plot the number of times the user stopped a session while being on this page, <br>
3.9) for each page plot the average time the user stayed on this page <br>
3.10) print the fist 10 sessions (paths). <br>

You may find the following classes helpful and data.

In [13]:
class Session:
    def __init__(self):
        self.startTime = 0
        self.stopTime = 0
        self.requests = []
        
    def __repr__(self):
        print("SESSION")
        print(self.startTime)
        print(self.stopTime)
        STR = ""
        for r in self.requests:
            print(r)
            STR += ("  " + r.page + "    " + str(datetime.timedelta(seconds=r.time)) + "\n")
        return STR

class Computer:
    def __init__(self):
        self.ip = []
        self.sessions = []
    def __repr__(self):
        return self.ip

PAGES = ["Shop.html", "AddProduct.html", "Summary.html", "Payment.html","About.html", "Contact.html"]

LINKS = [[1, 4],
         [2,4],
         [3],
         [],
         [1,5],
         []]

#plot distribution
#count, bins, ignored = plt.hist(data, 50, alpha=0.5, normed=True)
#plt.plot()

#x_axis = np.arange(len(pages))
#plt.bar(x_axis, data, align='center', alpha=0.5)
#plt.xticks(y_pos, pages, rotation=90)